In [ ]:
import time

from squid_py.config import Config
from squid_py.ocean.ocean import Ocean
from squid_py.keeper.web3_provider import Web3Provider
from ocean_cli.api.conditions import access_release
from ocean_cli.ocean import get_default_account

config = Config(filename='./bob.ini')
ocean = Ocean(config)
account = get_default_account(config)
    
template = ocean.keeper.escrow_access_secretstore_template.get_instance()
filter = template.events.AgreementCreated.createFilter(fromBlock='latest')

while True:
    for event in filter.get_new_entries():
        agreement_id = Web3Provider.get_web3().toHex(event['args']['_agreementId'])
        access_provider = event['args']['_accessProvider']
        access_consumer = event['args']['_accessConsumer']    
        if access_provider == account.address:
            access_release(ocean, account, agreement_id, access_consumer)
    time.sleep(0.5)

Using default logging settings.


2019-06-21 15:14:24 dimi-xps squid_py.keeper.diagnostics[14344] INFO Keeper contract artifacts (JSON abi files) at: /home/dimi/oceanprotocol/ocean-cli-py/artifacts
2019-06-21 15:14:24 dimi-xps squid_py.keeper.diagnostics[14344] INFO Using keeper contracts from network spree, network id is 8996
2019-06-21 15:14:24 dimi-xps squid_py.keeper.diagnostics[14344] INFO Looking for keeper contracts ending with ".spree.json", e.g. "AgreementStoreManager.spree.json".
2019-06-21 15:14:24 dimi-xps root[14344] INFO Finished loading keeper contracts:
	Dispenser: 0xF152cF3c67dFD41a317eAe8fAc0e1e8E98724A13
	OceanToken: 0x726baA2f854A3BEC2378a707AeB38c9d933Ebad6
	DIDRegistry: 0xc354ba9AD5dF1023C2640b14A09E61a500F21546
	AgreementStoreManager: 0x645439117eB378a6d35148452E287a038666Ed67
	TemplateStoreManager: 0xfeA10BBb093d7fcb1EDf575Aa7e28d37b9DcFcE9
	ConditionStoreManager: 0x336EFb3c9E56F713dFdA4CDB3Dd0882F3226b6eE
	AccessSecretStoreCondition: 0x19513460bc16254c74AE806683E906478A42B543
	SignCondition: 0x

Access:0x7183fa1e03774376b8c06ad4fa8236c832bfb9908e21437ca283cd41d24f165b-0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e True
Release:0x7183fa1e03774376b8c06ad4fa8236c832bfb9908e21437ca283cd41d24f165b True
Access:0xd482d577967a426696e625af96ea67a9de95f0628362467bae353b1cff444d15-0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e True
Release:0xd482d577967a426696e625af96ea67a9de95f0628362467bae353b1cff444d15 True
